In [1]:
import math
import datasets
import IBM1 as ibm1
import debug_helpers
import convergence_criterion as cc
import persistence
import plots

# pretty print variabeles on line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def fname_ibm1(i):
    return f'ibm1_iter_{i}.txt'

# After each iteration we print llhood and AER and store the model 
# We can later load the models that meet our convergence criteria
# and apply those to the test data
def fn_after_iter_ibm1(i, lprobs, log_likelihood, AER):
    debug_helpers.print_likelihood(i, lprobs, log_likelihood, AER)
    if i > 0:
        persistence.store_ibm1_model(lprobs, fname_ibm1(i))
        
# Run EM on training data set with AER on validation set
def IBM1_Train():
    val_sentence_pairs, _, _ = datasets.validation_data()
    reference_alignments = datasets.validation_alignments()    
    s_t_pairs, s_vocabulary, t_vocabulary = datasets.training_data()
    (lprobs, log_lhoods, AERs) = ibm1.EM(
        s_t_pairs, s_vocabulary, t_vocabulary, 16,
        val_sentence_pairs, reference_alignments, 
        fn_after_iter_ibm1
    ) 
    return (lprobs, log_lhoods, AERs)

In [ ]:
(lprobs, llhoods, AERs) = IBM1_Train()
plots.figure_log_likelihood(llhoods, 'IBM 1')
plots.figure_AER(AERs, 'IBM 1')

selected_model_AER = fname_ibm1(cc.select_model_AER(AERs))
selected_model_llhood = fname_ibm1(cc.select_model_LLhood(llhoods))
print('model selected on AER:', selected_model_AER)
print('model selected on LLhood:', selected_model_llhood)

# AER_model = persistence.load_ibm1_model(selected_model_AER)
# llhood_model = persistence.load_ibm1_model(selected_model_llhood)
# TODO: apply selected models to test set 
# (write NAACL and use official tool)